# Create training data for AutoCellLabeler

This notebook takes as input the following directories:

- `path_rgb` contains RGB images for NeuroPAL aniamls containing mNeptune, OFP, and BFP images. Each image should be an `X*Y*Z*3` matrix. In the directory, each image should be saved as `dataset.nrrd`.
- `path_tagrfp` contains TagRFP (all-red) images for the same animals, saved as `dataset.nrrd`. 
- `path_roi` contains ROI images for the same animals, saved as `dataset.nrrd`.
- `path_label` contains human-labeled CSV files for the same animals, saved as `dataset Neuron ID.csv`.
- `path_neuron_ids_file` is an `h5` file that consists of all valid labels. Labels other than these are ignored in the datasets.
- `path_output` contains the root path for outputs. These outputs will be the properly-formatted training, validation, and testing data for AutoCellLabeler.

The `path_rgb`, `path_tagrfp`, and `path_roi` images for a dataset respectively correspond to the `NeuroPAL.nrrd`, `all_red.nrrd`, and `neuron_rois.nrrd` files output by the ANTSUN notebook for that dataset.

It then crops the input images and formats them for input into AutoCellLabeler as training, validation, and testing data. It also crops the ROI images the same way.

The output of this notebook is also used to generate CellDiscoveryNet training data - the `make_CellDiscoveryNet_input.ipynb` notebook takes as input the output of this notebook. Notably, that notebook does not use any of the human labels as CellDiscoveryNet uses unsupervised learning. You can add blank CSV label files and run this notebook with them if you want to use additional unlabeled images to help train CellDiscoveryNet.

In [ ]:
import os

import pandas as pd 

import nrrd
import numpy as np

import h5py

from tqdm import tqdm

from matplotlib import pyplot as plt

import openpyxl
import csv
import re

import itertools

from functools import reduce

import autolabel

In [ ]:
path_rgb = "/data3/adam/new_unet_train/NeuroPAL_rgb_images/"
path_tagrfp = "/data3/adam/new_unet_train/NeuroPAL_all_red_images/"
path_roi = "/data3/adam/new_unet_train/NeuroPAL_roi_images/"
path_label = "/data3/adam/new_unet_train/csv_paper_1/"
path_neuron_ids_file = "/data3/adam/new_unet_train/extracted_neuron_ids_final_1.h5"

path_output = "/data3/adam/new_unet_train/"

path_train = "train" # subdirectory of `path_output` to put training data
path_val = "val" # subdirectory of `path_output` to put validation data
path_test = "test" # subdirectory of `path_output` to put testing data
path_roi_crop = "roi_crop" # subdirectory of `path_output` to put cropped ROI images

crop_size = (64, 120, 284) # size of the cropped images (Z, Y, X)

### Specify worm orientation

The worm can lie in two possible orientations, represented by `True` and `False` values in the `θh_pos_is_ventral` dictionary. Due to optical properties being different between the orientations, it is important to balance the frequency of each orientation between training, validation, and testing data. This notebook will also rotate all images to be in the same orientation. For AutoCellLabeler this does not matter as the network's data augmentation will randomize the orientation. However, for CellDiscoveryNet this is very important.

In [2]:
θh_pos_is_ventral = {
    "2023-06-09-01": True,
    "2023-06-24-02": False,
    "2023-06-24-28": True,
    "2023-07-01-01": True,
    "2023-07-01-09": False,
    "2023-07-07-01": False,
    "2023-07-07-18": True,
    "2023-07-11-02": False,
    "2023-07-28-04": True,
    "2023-07-07-11": False,
    "2023-07-12-01": True,
    "2023-08-07-01": False,
    "2023-08-22-08": True,
    "2023-08-18-11": False,
    "2023-06-24-11": True,
    "2023-07-13-01": False,
    "2023-08-07-16": False,
    "2023-06-09-10": True,
    "2023-08-22-01": False,
    "2023-08-23-23": False,
    "2023-08-25-02": True,
    "2023-09-15-01": True,
    "2023-09-15-08": True,
    "2023-08-18-18": False,
    "2023-08-19-01": True,
    "2023-08-23-09": True,
    "2023-09-02-10": True,
    "2023-08-25-09": False,
    "2023-09-01-01": True,
    "2023-08-31-03": False,
    "2023-07-01-23": False,
    "2021-05-26-07": True,
    "2021-06-11-01": True,
    "2021-08-04-06": False,
    "2021-08-17-01": True,
    "2021-08-18-01": True,
    "2021-09-06-09": True,
    "2021-09-14-01": True,
    "2021-09-14-05": False,
    "2021-09-22-05": True,
    "2021-09-23-01": True,
    "2021-09-30-01": False,
    "2021-10-26-01": False,
    "2021-11-12-01": True,
    "2021-11-12-05": False,
    "2022-01-07-03": True, # NOT ACTUALLY COMPUTED
    "2022-01-09-01": False,
    "2022-01-17-01": False,
    "2022-01-23-01": True,
    "2022-01-26-01": True,
    "2022-01-27-01": False,
    "2022-01-27-04": True,
    "2022-02-08-01": True,
    "2022-02-08-04": False,
    "2022-02-16-01": False,
    "2022-02-16-04": True,
    "2022-03-15-04": True,
    "2022-03-16-01": True, # NOT ACTUALLY COMPUTED
    "2022-03-16-02": True, # NOT ACTUALLY COMPUTED
    "2022-03-22-01": True,
    "2022-04-05-01": True,
    "2022-04-12-04": True,
    "2022-04-14-04": True,
    "2022-04-18-04": False,
    "2022-05-17-01": False,
    "2022-05-17-06": False,
    "2022-05-25-02": False,
    "2022-06-14-01": True,
    "2022-06-14-07": True,
    "2022-06-14-13": True,
    "2022-06-28-01": True,
    "2022-06-28-07": True,
    "2022-07-15-06": True,
    "2022-07-15-12": True,
    "2022-07-20-01": True,
    "2022-07-26-01": True,
    "2022-07-29-08": True,
    "2022-08-02-01": True,
    "2022-12-21-06": True,
    "2023-01-05-01": True,
    "2023-01-05-18": True,
    "2023-01-06-01": True,
    "2023-01-06-08": True,
    "2023-01-06-15": True,
    "2023-01-09-08": True,
    "2023-01-09-15": True,
    "2023-01-09-22": True,
    "2023-01-09-28": True,
    "2023-01-10-07": True,
    "2023-01-10-14": True,
    "2023-01-13-07": True,
    "2023-01-16-01": True,
    "2023-01-16-08": True,
    "2023-01-16-15": True,
    "2023-01-16-22": True,
    "2023-01-17-01": True,
    "2023-01-17-07": True,
    "2023-01-17-14": True,
    "2023-01-18-01": True,
    "2023-01-19-01": False,
    "2023-01-19-08": True,
    "2023-01-19-15": False,
    "2023-01-19-22": True,
    "2023-01-23-01": True,
    "2023-01-23-08": True,
    "2023-01-23-15": True,
    "2023-01-23-21": True,
    "2023-03-07-01": True,
    '2022-07-26-31': True,
    '2022-07-26-38': True,
    '2022-07-27-31': True,
    '2022-07-27-38': True,
    '2022-07-27-45': True,
    '2022-08-02-31': True,
    '2022-08-02-38': True,
    '2022-08-03-31': False,
    "2023-03-30-01": True,
    "2023-06-21-01": False,
    "2023-06-23-08": True,
    "2023-06-29-01": False,
    "2023-06-29-13": True,
    "2023-07-14-08": True,
    "2023-07-14-14": False,
    "2023-07-27-01": True,
    "2023-07-27-08": True,
    "2023-08-08-07": False,
    "2023-08-14-01": True,
    "2023-08-15-01": False,
    "2023-08-16-01": True,
    "2023-08-21-01": True,
    "2023-09-07-01": False,
    "2023-09-14-01": True,
    "2023-09-25-01": True,
    "2023-10-05-01": False,
    "2023-12-11-01": True,
    "2023-05-25-08": False,
    "2023-05-26-08": False,
    "2023-06-05-10": True,
    "2023-06-05-17": False,
    "2023-07-24-27": True,
    "2023-09-27-14": False,
    "2023-05-25-01": False,
    "2023-05-26-01": False,
    "2023-05-30-14": False,
    "2023-07-24-12": False,
    "2023-07-24-20": False,
    "2023-09-12-01": False,
    "2023-09-19-01": False,
    "2023-09-29-19": False,
    "2023-10-09-01": False,
    "2023-10-09-07": False,
    "2023-09-13-02": False
}

### Specify datasets

This code specifies which datasets you are using and how to allocate them between training, validation, and testing data. The settings shown by default are the same settings used to train AutoCellLabeler in the manuscript. However, after training, datasets `2023-08-15-01` and `2023-12-11-01` were found to have incorrect labels. For future AutoCellLabeler training runs, they should not be used.

Additionally, humans often make two mistakes when labeling data, which you may need to manually correct in the `csv` label files if importing new ones:

- `RMH` is not a valid label. `RMH` labels should be replaced with `RMH?`.
- `SABD?` is not a valid label. `SABD?` labels should be replaced with `SABD`.

In [5]:
datasets_prj_neuropal = ["2022-07-15-06", "2022-07-15-12", "2022-07-20-01", "2022-07-26-01", "2022-08-02-01", "2023-01-23-08", "2023-01-23-15", "2023-01-23-21", "2023-01-19-08", "2023-01-19-22", "2023-01-09-28", "2023-01-17-01", "2023-01-19-15", "2023-01-23-01", "2023-03-07-01", "2022-12-21-06", "2023-01-05-18", "2023-01-06-01", "2023-01-06-08", "2023-01-09-08", "2023-01-09-15", "2023-01-09-22", "2023-01-10-07", "2023-01-10-14", "2023-01-13-07", "2023-01-16-01", "2023-01-16-08", "2023-01-16-15", "2023-01-16-22", "2023-01-17-07", "2023-01-17-14", "2023-01-18-01"]
datasets_prj_rim = ["2023-06-09-01", "2023-07-28-04", "2023-06-24-02", "2023-07-07-11", "2023-08-07-01", "2023-06-24-11", "2023-07-07-18", "2023-08-18-11", "2023-06-24-28", "2023-07-11-02", "2023-08-22-08", "2023-07-12-01", "2023-07-01-09", "2023-07-13-01", "2023-06-09-10", "2023-07-07-01", "2023-08-07-16", "2023-08-22-01", "2023-08-23-23", "2023-08-25-02", "2023-09-15-01", "2023-09-15-08", "2023-08-18-18", "2023-08-19-01", "2023-08-23-09", "2023-08-25-09", "2023-09-01-01", "2023-08-31-03", "2023-07-01-01", "2023-07-01-23"]

datasets_prj_aversion = ["2023-03-30-01", "2023-06-29-01", "2023-06-29-13", "2023-07-14-08", "2023-07-14-14", "2023-07-27-01", "2023-08-08-07", "2023-08-14-01", "2023-08-16-01", "2023-08-21-01", "2023-09-07-01", "2023-09-14-01", "2023-08-15-01", "2023-10-05-01", "2023-06-23-08", "2023-12-11-01", "2023-06-21-01"]
datasets_prj_5ht = ["2022-07-26-31", "2022-07-26-38", "2022-07-27-31", "2022-07-27-38", "2022-07-27-45", "2022-08-02-31", "2022-08-02-38", "2022-08-03-31"]
datasets_prj_starvation = ["2023-05-25-08", "2023-05-26-08", "2023-06-05-10", "2023-06-05-17", "2023-07-24-27", "2023-09-27-14", "2023-05-25-01", "2023-05-26-01", "2023-07-24-12", "2023-07-24-20", "2023-09-12-01", "2023-09-19-01", "2023-09-29-19", "2023-10-09-01", "2023-09-13-02"]

datasets = datasets_prj_neuropal + datasets_prj_rim + datasets_prj_aversion + datasets_prj_5ht + datasets_prj_starvation
print(len(set(datasets)) == len(datasets))

datasets_val = ['2023-06-24-02', '2023-08-07-01', '2023-08-19-01', # RIM datasets
                '2022-07-26-01', '2023-01-23-21', '2023-01-23-01', # NeuroPAL datasets
                '2023-07-14-08', # Aversion datasets
                '2022-08-02-31', # 5-HT datasets
                '2023-07-24-27', '2023-07-24-20'] # Starvation datasets
datasets_test = ['2023-08-22-01', '2023-07-07-18', '2023-07-01-23',  # RIM datasets
                 '2023-01-06-01', '2023-01-10-07', '2023-01-17-07', # Neuropal datasets
                 '2023-08-21-01', "2023-06-23-08", # Aversion datasets
                 '2022-07-27-38', # 5-HT datasets
                 '2023-10-09-01', '2023-09-13-02' # Starvation datasets
                 ]
datasets_train = [dataset for dataset in datasets if dataset not in datasets_val and dataset not in datasets_test]

# there should be no overlapping datasets between train, val, test
assert(len([d for d in datasets_val if d in datasets_test]) == 0)
assert(len([d for d in datasets_train if d in datasets_val]) == 0)
assert(len([d for d in datasets_train if d in datasets_test]) == 0)

print("Number of train/val/test datasets:", len(datasets_train), len(datasets_val), len(datasets_test))

# all datasets should have orientation label
assert(len([d for d in datasets if not d in θh_pos_is_ventral]) == 0)

print("Number of train/val/test datasets in common orientation:", len([d for d in datasets_train if d in θh_pos_is_ventral and θh_pos_is_ventral[d]]), len([d for d in datasets_val if d in θh_pos_is_ventral and θh_pos_is_ventral[d]]), len([d for d in datasets_test if d in θh_pos_is_ventral and θh_pos_is_ventral[d]]))

# files to modify if data is copy/pasted
# RMH --> RMH? in all datasets
# SABD? --> SABD in all datasets

True
Number of train/val/test datasets: 81 10 11
Number of train/val/test datasets in common orientation:  51 7 7


### Load human labels and count rarity of labels

In [ ]:
dict_roi_to_neuron_all = {}

for dataset in tqdm(datasets):
    dict_roi_to_neuron_all[dataset] = map_roi_to_neuron(os.path.join(path_label, dataset + " Neuron ID.csv"))

extracted_neuron_ids = []
with h5py.File(path_neuron_ids_file, 'r') as f:
    extracted_neuron_ids = [name.decode('utf-8') for name in f['neuron_ids'][:]]

num_labels = {}
num_question_labels = {}

for dataset in tqdm(datasets):
    for roi in dict_roi_to_neuron_all[dataset][0].keys():
        for neuron in dict_roi_to_neuron_all[dataset][0][roi]:
            if neuron in num_labels:
                num_labels[neuron] += 1
            else:
                num_labels[neuron] = 1
            if '?' in neuron:
                for comb in generate_combinations(neuron):
                    if comb in num_question_labels:
                        num_question_labels[comb] += 1
                    else:
                        num_question_labels[comb] = 1


### Generate AutoCellLabeler training data

In [ ]:
for dataset in tqdm(datasets):
    if dataset in datasets_test:
        save_dir = path_test
    elif dataset in datasets_val:
        save_dir = path_val
    else:
        save_dir = path_train

    # create output directories
    full_output_path = os.path.join(path_output, save_dir)
    os.makedirs(full_output_path, exist_ok=True)
    full_roi_crop_path = os.path.join(path_output, path_roi_crop)
    os.makedirs(full_roi_crop_path, exist_ok=True)
    
    rgb_path = os.path.join(path_rgb, dataset + ".nrrd")
    output_path = os.path.join(full_output_path, dataset + ".h5")
    roi_path = os.path.join(path_roi, dataset + ".nrrd")
    roi_crop_path = os.path.join(full_roi_crop_path, dataset + ".h5")
    label_path = os.path.join(path_label, dataset + " Neuron ID.csv")
    all_red_path = os.path.join(path_tagrfp, dataset + ".nrrd")

    # the `id_weight` parameter currently has little effect on the output, it describes how much smaller the weights are for wrong labels (which currently does not matter)    
    create_h5_from_nrrd(rgb_path, output_path, roi_path, roi_crop_path, crop_size, num_labels, θh_pos_is_ventral[dataset], label_file=label_path, neuron_ids_list_file=path_neuron_ids_file, all_red_path=all_red_path, id_weight=0.3)
